In [ ]:
import streamlit as st
import plotly.express as px
import pandas as pd
import plotly.io as pio
import glob


pio.templates.default = None

pattern = "local_quote_data/autoUSD_*.csv"
files_with_glob = glob.glob(pattern)
print("With glob:", files_with_glob)

df = pd.read_csv(files_with_glob[0])
slippage_df = pd.read_csv(files_with_glob[1])
df

With glob: ['local_quote_data/autoUSD_quotes_1751571194.csv', 'local_quote_data/autoUSD_slippage_1751571194.csv']


,aggregatorName,asyncSwapper,internal,buyAmount,minBuyAmount,expiration,tx,fullQuoteDetails,chainId,systemName,...,token_address,chain_id,symbol,name,decimals,buy_amount_norm,min_buy_amount_norm,sell_amount_norm,ratio,min_buy_amount_ratio
0,4626,0x1dFB5041c3f594cAEF41bcEc02f3a9Af47Ae8c88,True,6.136028e+11,6.105347e+11,1.751571e+09,{'from': '0x0000000000000000000000000000000000...,"{'redeem': True, 'swap': True, 'underlyingQuot...",1,gen3,...,0x0655977FEb2f289A4aB78af67BAB0d17aAb84367,1,scrvUSD,Savings crvUSD,18,6.136028e+05,6.105347e+05,5.836038e+05,1.051403,1.046146
1,4626,0x1dFB5041c3f594cAEF41bcEc02f3a9Af47Ae8c88,True,1.227194e+12,1.221058e+12,1.751571e+09,{'from': '0x0000000000000000000000000000000000...,"{'redeem': True, 'swap': True, 'underlyingQuot...",1,gen3,...,0x0655977FEb2f289A4aB78af67BAB0d17aAb84367,1,scrvUSD,Savings crvUSD,18,1.227194e+06,1.221058e+06,1.167208e+06,1.051393,1.046136
2,4626,0x1dFB5041c3f594cAEF41bcEc02f3a9Af47Ae8c88,True,1.840707e+12,1.831503e+12,1.751571e+09,{'from': '0x0000000000000000000000000000000000...,"{'redeem': True, 'swap': True, 'underlyingQuot...",1,gen3,...,0x0655977FEb2f289A4aB78af67BAB0d17aAb84367,1,scrvUSD,Savings crvUSD,18,1.840707e+06,1.831503e+06,1.750811e+06,1.051345,1.046088
3,4626,0x1dFB5041c3f594cAEF41bcEc02f3a9Af47Ae8c88,True,2.454230e+12,2.441958e+12,1.751571e+09,{'from': '0x0000000000000000000000000000000000...,"{'redeem': True, 'swap': True, 'underlyingQuot...",1,gen3,...,0x0655977FEb2f289A4aB78af67BAB0d17aAb84367,1,scrvUSD,Savings crvUSD,18,2.454230e+06,2.441958e+06,2.334415e+06,1.051325,1.046069
4,4626,0x1dFB5041c3f594cAEF41bcEc02f3a9Af47Ae8c88,True,3.067758e+12,3.052419e+12,1.751571e+09,{'from': '0x0000000000000000000000000000000000...,"{'redeem': True, 'swap': True, 'underlyingQuot...",1,gen3,...,0x0655977FEb2f289A4aB78af67BAB0d17aAb84367,1,scrvUSD,Savings crvUSD,18,3.067758e+06,3.052419e+06,2.918019e+06,1.051315,1.046059
5,4626,0x1dFB5041c3f594cAEF41bcEc02f3a9Af47Ae8c88,True,1.051430e+06,1.046173e+06,1.751571e+09,{'from': '0x0000000000000000000000000000000000...,"{'redeem': True, 'swap': True, 'underlyingQuot...",1,gen3,...,0x0655977FEb2f289A4aB78af67BAB0d17aAb84367,1,scrvUSD,Savings crvUSD,18,1.051430e+00,1.046173e+00,1.000000e+00,1.051430,1.046173
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,gen3,...,0x0655977FEb2f289A4aB78af67BAB0d17aAb84367,1,scrvUSD,Savings crvUSD,18,NaN,NaN,0.000000e+00,NaN,NaN
7,Odos,0x8ea340AA42cD1f6e2471a5C4574EA62C9416B859,False,8.532244e+11,8.489583e+11,1.000000e+10,{'from': '0x0000000000000000000000000000000000...,{'quote': {'traceId': '36e508ac-c4d2-45f9-a1a9...,1,gen3,...,0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f,1,GHO,Gho Token,18,8.532244e+05,8.489583e+05,8.537133e+05,0.999427,0.994430
8,0xV2,0x0AB82316aD6e206Cab38EF9D36bD8cAEa86Bc18B,False,1.706416e+12,1.697884e+12,1.000000e+10,{'from': '0x0000000000000000000000000000000000...,"{'blockNumber': '22840809', 'buyAmount': '1706...",1,gen3,...,0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f,1,GHO,Gho Token,18,1.706416e+06,1.697884e+06,1.707427e+06,0.999408,0.994411
9,0xV2,0x0AB82316aD6e206Cab38EF9D36bD8cAEa86Bc18B,False,2.559582e+12,2.546784e+12,1.000000e+10,{'from': '0x0000000000000000000000000000000000...,"{'blockNumber': '22840809', 'buyAmount': '2559...",1,gen3,...,0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f,1,GHO,Gho Token,18,2.559582e+06,2.546784e+06,2.561140e+06,0.999392,0.994395


In [ ]:
px.scatter(
    slippage_df,
    x="sell_amount_norm",
    y="ratio",
    color="symbol",
    title="Sell amount to ratio sell / buy amount ratio",
)
# selling 1 is not a fair comparison because we some t

In [19]:
px.scatter(
    slippage_df,
    x="sell_amount_norm",
    y="buy_amount_norm",
    color="symbol",
    title="Sell amount to ratio sell / buy amount ratio",
)
# selling 1 is not a fair comparison because we some t

In [10]:
usdt_df = df[df["symbol"] == "USDT"].copy()
usdt_df

,aggregatorName,asyncSwapper,internal,buyAmount,minBuyAmount,expiration,tx,fullQuoteDetails,chainId,systemName,...,token_address,chain_id,symbol,name,decimals,buy_amount_norm,min_buy_amount_norm,sell_amount_norm,ratio,min_buy_amount_ratio
30,Odos,0x8ea340AA42cD1f6e2471a5C4574EA62C9416B859,False,194730680594,193757027192,1.000000e+10,{'from': '0x0000000000000000000000000000000000...,{'quote': {'traceId': 'e72892a0-3333-4cd6-9895...,1,gen3,...,0xdAC17F958D2ee523a2206206994597C13D831ec7,1,USDT,Tether USD,6,194730.680594,193757.027192,194655.643823,1.000385,0.995384
31,0xV2,0x0AB82316aD6e206Cab38EF9D36bD8cAEa86Bc18B,False,389460126846,387512819400,1.000000e+10,{'from': '0x0000000000000000000000000000000000...,"{'blockNumber': '22840713', 'buyAmount': '3894...",1,gen3,...,0xdAC17F958D2ee523a2206206994597C13D831ec7,1,USDT,Tether USD,6,389460.126846,387512.819400,389311.287647,1.000382,0.995380
32,0xV2,0x0AB82316aD6e206Cab38EF9D36bD8cAEa86Bc18B,False,584188338766,581267388350,1.000000e+10,{'from': '0x0000000000000000000000000000000000...,"{'blockNumber': '22840713', 'buyAmount': '5841...",1,gen3,...,0xdAC17F958D2ee523a2206206994597C13D831ec7,1,USDT,Tether USD,6,584188.338766,581267.388350,583966.931471,1.000379,0.995377
33,Odos,0x8ea340AA42cD1f6e2471a5C4574EA62C9416B859,False,778915316378,775020739797,1.000000e+10,{'from': '0x0000000000000000000000000000000000...,{'quote': {'traceId': 'ef5d4d58-e6a0-4159-af93...,1,gen3,...,0xdAC17F958D2ee523a2206206994597C13D831ec7,1,USDT,Tether USD,6,778915.316378,775020.739797,778622.575295,1.000376,0.995374
34,0xV2,0x0AB82316aD6e206Cab38EF9D36bD8cAEa86Bc18B,False,973641059658,968772844750,1.000000e+10,{'from': '0x0000000000000000000000000000000000...,"{'blockNumber': '22840713', 'buyAmount': '9736...",1,gen3,...,0xdAC17F958D2ee523a2206206994597C13D831ec7,1,USDT,Tether USD,6,973641.059658,968772.844750,973278.219119,1.000373,0.995371
35,Odos,0x8ea340AA42cD1f6e2471a5C4574EA62C9416B859,False,1020390,1015289,1.000000e+10,{'from': '0x0000000000000000000000000000000000...,{'quote': {'traceId': 'ef6eb20d-8cab-4929-bd2b...,1,gen3,...,0xdAC17F958D2ee523a2206206994597C13D831ec7,1,USDT,Tether USD,6,1.020390,1.015289,1.000000,1.020390,1.015289


In [ ]:
# usdt quotes are bad

In [11]:
df.columns

Index(['aggregatorName', 'asyncSwapper', 'internal', 'buyAmount',
       'minBuyAmount', 'expiration', 'tx', 'fullQuoteDetails', 'chainId',
       'systemName', 'slippageBps', 'taker', 'sellToken', 'buyToken',
       'sellAmount', 'includeSources', 'excludeSources', 'sellAll',
       'timeoutMS', 'transferToCaller', 'same_token', 'token_address',
       'chain_id', 'symbol', 'name', 'decimals', 'buy_amount_norm',
       'min_buy_amount_norm', 'sell_amount_norm', 'ratio',
       'min_buy_amount_ratio'],
      dtype='object')

In [13]:
sUSDe_df[["buy_amount_norm", "sell_amount_norm", "ratio", "symbol", "buyToken", "sellToken"]]

,buy_amount_norm,sell_amount_norm,ratio,symbol,buyToken,sellToken
18,29861.360610,25263.017910,1.182019,sUSDe,0x865377367054516e17014CcdED1e7d814EDC9ce4,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497
19,59721.915051,50526.035820,1.182003,sUSDe,0x865377367054516e17014CcdED1e7d814EDC9ce4,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497
20,89581.545760,75789.053729,1.181985,sUSDe,0x865377367054516e17014CcdED1e7d814EDC9ce4,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497
21,119440.492728,101052.071639,1.181970,sUSDe,0x865377367054516e17014CcdED1e7d814EDC9ce4,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497
22,149299.353377,126315.089549,1.181960,sUSDe,0x865377367054516e17014CcdED1e7d814EDC9ce4,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497
23,1.232677,1.000000,1.232677,sUSDe,0x865377367054516e17014CcdED1e7d814EDC9ce4,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497
